<h1> Spotify Skip Prediction Dataset </h1>
This dataset comes in two sets. The first set is details about 'sessions': chunks of songs a user listens to in one go, and what songs were listened to. The second set details the song's features. <br>
Our analysis will include just the mini set availible on AI Crowd. The input we are using is an augmented table that combines the user session data and the song features data. 
There are 167880 entires and 50 total features. Only 47 features will be used. 

<h3>References</h3>
We'd like to recognize that due to enormity of this dataset and the complexity of how it was stored (in multiple seperate and unorganized .csv files), we did use online references to decide on our stack and how we would approach the data. <br>
We used the following a examples: <br> <br>
<li> <a>https://github.com/a-poor/spotify-skip-prediction/blob/master/README.md</a>
<br><i>Used for template tech stack and reorganize dataset. </i>
<li> <a>https://www.youtube.com/watch?v=KuQUNHCeKCk&ab_channel=BekBrace </a>
<br><i>Used this series to conigure PostgreSQL.</i>

<h2>Import Libraries and Datasets</h2>

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# load both csv files into dataframes
log_df = pd.read_csv('log_mini.csv') # user log
tf_df  = pd.read_csv('tf_mini.csv')  # track features
 
# rename and merge the two data frames so that the 
log_df = log_df.rename(columns={'track_id_clean': 'track_id'})

# perform a merge so that song information is attached to the user information
og_data_df = pd.merge(log_df, tf_df, on='track_id')

# Save the merged DataFrame to a new CSV file
og_data_df.to_csv('merged_file.csv', index=False)
data_df = og_data_df

In [46]:
print(og_data_df.head(1))
print(data_df.shape)

                               session_id  session_position  session_length  \
0  0_00006f66-33e5-4de7-a324-2d18e439fc1e                 1              20   

                                 track_id  skip_1  skip_2  skip_3  \
0  t_0479f24c-27d2-46d6-a00c-7ec928f2b539   False   False   False   

   not_skipped  context_switch  no_pause_before_play  ...  time_signature  \
0         True               0                     0  ...               4   

    valence  acoustic_vector_0  acoustic_vector_1  acoustic_vector_2  \
0  0.152255          -0.815775           0.386409            0.23016   

   acoustic_vector_3 acoustic_vector_4  acoustic_vector_5 acoustic_vector_6  \
0           0.028028         -0.333373           0.015452          -0.35359   

  acoustic_vector_7  
0          0.205826  

[1 rows x 50 columns]
(167880, 50)


# Reorganize Data
We decided to select the the variable skip_3 as our 'y' variable. The 'skip_3' feautre represents when a 

In [30]:
# turn skip_3 from T/F to binary(0, 1)
data_df['skip_3'] = data_df['skip_3'].astype(int)

# drop 'skip_3' from the data and make it our 'y' value
y_df = data_df['skip_3']

data_df = data_df.drop(columns=["skip_1", "skip_2", "skip_3", "not_skipped"], axis=1)

In [41]:
# check for missing values
missing_values = data_df.isnull().sum()

# display columns with missing values and counts
print(missing_values[missing_values > 0])

Series([], dtype: int64)


In [42]:
X_train, X_test, y_train, y_test = train_test_split(data_df, y_df, test_size=0.2, random_state=42)

In [48]:
value_counts = data_df['session_id'].value_counts()

# Display the counts of each unique value in the column
print(value_counts)

session_id
0_00006f66-33e5-4de7-a324-2d18e439fc1e    20
0_08317754-b299-481b-ae44-e74b4b232ac2    20
0_0821a0cd-2a85-4ceb-8c11-a1a4748e74d8    20
0_08202dbe-7375-49eb-a793-2d1579fb3afd    20
0_081fc470-cea4-4a3f-a238-22099654c153    20
                                          ..
0_088b55d4-9da8-4237-91f0-5579d811986b    10
0_030d00a4-de73-4697-a1a6-b765e231f80d    10
0_0d207bfd-43fe-4b1e-8c8f-736536555140    10
0_030ea191-5c2c-45ae-8ae7-a1875adabe86    10
0_084483a5-5562-4305-bc68-5687110700f5    10
Name: count, Length: 10000, dtype: int64


# Model #1: Linear Regression
Our first model will be a linear regression model using sckit-learn. 


In [44]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [45]:
model = LinearRegression()
model.fit(X_train, y_train)

ValueError: could not convert string to float: '0_0d50c3c5-ea8e-41c5-9d0d-d474a478f75d'